In [2]:
from pathlib import Path
import shutil

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import locale
#from html.parser import HTMLParser
# import website_func.py to use its functions
from website_func import *
import os #To read the file
#reload every module each time
%load_ext autoreload
%autoreload 2

# Some quick analysis of the data
We received a data set of .html file containing the website content of recipes.
We wanted to sort them by website, in order to, more easily, find a pattern among them. This will enable us to do the "scraping" of the pages. First we thought about moving the files in a folder corresponding to their website, but it would be a waste of time and a big computational effort. Thus, we came up with a (probably) faster solution : we could simply write the name of the file within its corresponding website folder. By inspecting the files, we saw that the first line was always containing a comment with the name of the file and the complete website. Using readlines and split, we could easily retrieve the name of the website.

We launched this process, but an alarm appeared describing a Trojan virus in the file "1c2cb6f0df04cf5a9d0baa116c6aa7bb.html". 
We had then to quarantine or mabye remove the file, as we have quite enough
By doing so, we remarked the file "msg.log" that could help us into fastering the processus as its content is formed of the name of the file together with its website. [SHOW] We obserbed that occasionally a line containing other info that the content we want can appear. We need to ignore this
Also we noticed there are no file extensions other than .html and .log. 

In [14]:
retrieve_website_from_log()

FileNotFoundError: [Errno 2] No such file or directory: 'recipePages.zip\\recipePages\\msg.log'

In [4]:
# We can find in the folder that, excepted the html files, there is only the log

# Get all filenames (i.e. path) that are in recipePages folder 
pathlist = Path("recipePages/").glob('**/*')
i = 0
for path in pathlist:
     # because path is object not string
    path_in_str = str(path)
    if not path_in_str.endswith(".html"):
        print(path_in_str)

recipePages\msg.log


In [5]:
website_count=pd.Series()

for (root,dirs,files) in os.walk('SortedFiles', topdown="True"):   
        for website in dirs:
            
            filename = "SortedFiles/" + website +"/filesName.txt"
            
            f = open(filename, "r")
            number_of_link = len(f.readlines())
            line_to_add=pd.Series(number_of_link,index=[website])
            website_count = website_count.append(line_to_add)

FileNotFoundError: [Errno 2] No such file or directory: 'SortedFiles/.ipynb_checkpoints/filesName.txt'

In [6]:
website_count.sort_values(ascending=False,inplace=True)

In [7]:
website_count_used=website_count[website_count.values>1000]

In [8]:
website_list_used=website_count_used.index.tolist()

In [238]:
#Words that have to be removed from the string in the website to keep only the ingredient:
list_ingredient_to_remove = ["teaspoon" , "cup", "cups","teaspoons","ounce","ounces","pounds","pound","tablespoons","chopped","quarts","fresh",\
                            "light","plain","popped","medium","shredded","bunch","tablespoon","to","taste","pinch","freshly","ground",\
                            "canned","carcass","cubes","dried","frozen","all","purpose","grated","minced","degree","degrees","optional",\
                            "jar","quartered","marinated","strips","strip","asian","stalks","package","can","cans","container",\
                            "evaporated","bottle","cans","extract","squares","german","slices","crushed","uncooked","seasoning","small",\
                            "sweet","packed","sliced","heavy","condensed","finely","long","grain","sweetened","firmly","cooked","crusts","lean",\
                            "caps","mix","steaks","large","instant","crumbs","semisweet","distilled","packages","pint","miniature","preserves",\
                            "processed","chunky","prepared","seasoned","american","stock","top","sirloin","tip","peeled","spears","leaves",\
                            "belgian","bob","italian","bottles","boneless","roast","breast","meat","tenderloins","granules","chips","converted",\
                            "containers","coleslaw","florets","cube","cubed","diced","crumbled"]

list_ingredient_to_remove = sorted(list_ingredient_to_remove)
print(list_ingredient_to_remove)

['all', 'american', 'asian', 'belgian', 'bob', 'boneless', 'bottle', 'bottles', 'breast', 'bunch', 'can', 'canned', 'cans', 'cans', 'caps', 'carcass', 'chips', 'chopped', 'chunky', 'coleslaw', 'condensed', 'container', 'containers', 'converted', 'cooked', 'crumbled', 'crumbs', 'crushed', 'crusts', 'cube', 'cubed', 'cubes', 'cup', 'cups', 'degree', 'degrees', 'diced', 'distilled', 'dried', 'evaporated', 'extract', 'finely', 'firmly', 'florets', 'fresh', 'freshly', 'frozen', 'german', 'grain', 'granules', 'grated', 'ground', 'heavy', 'instant', 'italian', 'jar', 'large', 'lean', 'leaves', 'light', 'long', 'marinated', 'meat', 'medium', 'minced', 'miniature', 'mix', 'optional', 'ounce', 'ounces', 'package', 'packages', 'packed', 'peeled', 'pinch', 'pint', 'plain', 'popped', 'pound', 'pounds', 'prepared', 'preserves', 'processed', 'purpose', 'quartered', 'quarts', 'roast', 'seasoned', 'seasoning', 'semisweet', 'shredded', 'sirloin', 'sliced', 'slices', 'small', 'spears', 'squares', 'stalks

In [ ]:
#Create the data from the information found on the HTML pages

recipe_data = pd.DataFrame(columns = ['Website','Recipe', 'Ranking', 'Reviews', 'Ingredients'])  

for (root,dirs,files) in os.walk('recipePagesT', topdown="True"):  
    for webpage in files:
        filename= 'recipePagesT/'+webpage
        
        #Try to open it, and if it doesn't, we don't analyse this webpage
        try:
            f = open(filename,'r')
            first_line=f.readline()
            f.close()
        except:
            print("We can't read the page:",filename)
            continue

        website = str(first_line).split("/")[2].strip("www.")
        
        #Determine if we have to spare the data
        if website in website_list_used :
            
            #Read the data from the first website (allrecipes.com)
            if website in website_list_used[0]:
                #print(filename)
                try:
                    soup = BeautifulSoup(open(filename), "html.parser")
                except:
                    print("Beautifulsoup can't read the page:",filename)
                    continue
                
                recipe_name = soup.find('span', class_='itemreviewed')
                
                #We only take recipes and not searching pages
                if recipe_name is None:
                    print("We don't care about this page: ",filename)
                    continue
                recipe_name = recipe_name.text
                
                rating_html = soup.find('img', class_='rating')
                
                #Determine position of the ranking in the string
                start_rank = 59
                end_rank = 62
                rating = rating_html['title'][start_rank:end_rank]
                
                #print(recipe_name)
                #print(rating)
                
                #Determine the number of reviews:
                if soup.find('span', class_='count') is not None:
                    review_html = soup.find('span', class_='count').text
                    review = int(review_html.replace(',',''))
                else:
                    review = 0
                #print('Reviews :',review)
                
                #Find the preparation time:
                prepare_time = np.nan
                soup1 = soup.find('span', class_='totalTime')
                if soup1 is not None:
                    
                    prepare_time_html = soup1.find('span', class_='value-title')
                    #Determine position of the time in the string
                    #print(prepare_time_html['title'])
                    start_prepare_time = 2
                    end_prepare_time = len(prepare_time_html['title']) 
                    prepare_time_not_converted = prepare_time_html['title'][start_prepare_time:end_prepare_time]
                    time_analyse = prepare_time_not_converted.split('H')
                    
                    #To convert into minutes
                    if len(time_analyse) == 1:
                        prepare_time = int(time_analyse[0].replace('M',''))
                    if len(time_analyse) == 2:
                        time_minute = time_analyse[1].replace('M','');
                        prepare_time = int(time_analyse[0])*60 + (int(time_minute) if time_minute else 0) 
                    
                    
                #print(type(prepare_time))
                
                #Find the ingredient list:
                list_ingred = []
                ingredients = soup.find('div', class_='ingredients')
                all_ingredients = ingredients.find_all('li',class_="plaincharacterwrap ingredient")
                for ingredient in all_ingredients:
                    ingredient_i = ingredient.text.replace('\n','').lower()
                    #print('Original:', ingredient_i)
                    ingredient_i = ingredient_i.split(',')[0]
                    ingredient_i = [word for word in ingredient_i.split(' ') if word not in list_ingredient_to_remove]  #Clean string to only have the ingredient
                    ingredient_i = ' '.join(x for x in ingredient_i if x.isalpha()) + ' '
                    ingredient_i = ingredient_i.replace(' or ', '//').replace(' and ','//').replace(' with ','//').split('//')   #if options, add both in the list of ingredients
                    #print('After removing:              ', ingredient_i,"\n")
                    
                    for ingredient_in_list in ingredient_i:
                        ingredient_in_list_strip = ingredient_in_list.strip()
                        if ingredient_in_list_strip == '':
                            continue
                        list_ingred.append(ingredient_in_list_strip)
                    
                #print(list_ingred)    
                
                recipe_data = recipe_data.append({'Website': website_list_used[0], 'Recipe': recipe_name, 'Ranking': rating, 'Reviews': review,\
                                                  'Ingredients': list_ingred}, ignore_index=True)
                
                
                #print('\n')
                

In [211]:
for data in recipe_data['Ingredients']:
    print(data)

['whole wheat', 'white flour', 'salt', 'white pepper', 'black pepper', 'chicken breast tenderloins', 'canola oil', 'olive oil', 'chicken broth', 'lemon juice', 'artichoke hearts', 'capers', 'butter', 'parsley']
['corn syrup', 'margarine', 'cold water', 'sugar', 'marshmallows', 'popcorn']
['cream cheese', 'butter', 'orange zest', 'orange juice', 'sugar', 'walnuts']
['baking potato', 'olive oil', 'salt', 'butter', 'black pepper', 'cheddar cheese']
['rolled oats', 'milk', 'pumpkin puree', 'pumpkin pie spice', 'cinnamon sugar']
['asparagus', 'cooking spray', 'salt', 'pepper', 'butter', 'soy sauce', 'balsamic vinegar']
['chicken', 'bay leaf', 'butter', 'onion', 'shallots', 'chicken bouillon', 'parsley', 'salt', 'pepper', 'potatoes', 'corn kernels', 'cream']
['baking flour', 'garbanzo bean flour', 'cornstarch', 'tapioca starch', 'baking powder', 'xanthan gum', 'italian', 'oregano', 'salt', 'parmesan cheese', 'white sugar', 'lukewarm water', 'active dry yeast', 'egg', 'olive oil', 'apple cide

In [235]:
list_all_ingredients = []
for data in recipe_data['Ingredients']:
    #print(data)    
    #ingredients_strip = data.strip()
    for i in range(len(data)):
        list_all_ingredients.append(data[i])
    
    
print(len(list_all_ingredients))
print(len(set(list_all_ingredients)))
set(list_all_ingredients)



872
341


{'acorn squash',
 'active dry yeast',
 'allspice',
 'american cheese',
 'and sour',
 'and wild rice',
 'angel food cake',
 'apple',
 'apple cider vinegar',
 'apple juice',
 'apples peeled',
 'artichoke hearts',
 'asparagus',
 'asparagus spears',
 'avocados peeled',
 'bacon',
 'baking flour',
 'baking potato',
 'baking powder',
 'baking soda',
 'balsamic vinegar',
 'banana',
 'barbeque sauce',
 'basil',
 'basil leaves',
 'basil sauce',
 'bay leaf',
 'bay leaves',
 'beef',
 'beef bouillon',
 'beef broth',
 'beef sirloin tip',
 'beef stock',
 'beef top sirloin',
 'belgian endives',
 'biscuit baking',
 'black drain',
 'black pepper',
 'bob italian sausage',
 'boiling water',
 'bok choy',
 'boneless pork shoulder roast',
 'bottles chile sauce',
 'box dry bread stuffing',
 'bread',
 'bread flour',
 'broccoli',
 'broccoli coleslaw',
 'broccoli florets',
 'brown rice',
 'brown sugar',
 'brownie',
 'butter',
 'buttermilk',
 'cajun',
 'cannellini beans',
 'canola oil',
 'capers',
 'carrots',
 'c

In [169]:
"for","frying","diced","thawed"

recipe_data

,Website,Recipe,Ranking,Reviews,Ingredients
0,allrecipes.com,Chicken Breast Cutlets with Artichokes and Capers,4.4,80,"[whole wheat, white flour, salt, white pepper,..."
1,allrecipes.com,Best Ever Popcorn Balls,4.4,322,"[corn syrup, margarine, cold water, sugar, mar..."
2,allrecipes.com,Orange Cream Cheese Frosting,4.6,39,"[cream cheese, butter, orange zest, orange jui..."
3,allrecipes.com,Perfect Baked Potato,4.7,410,"[baking potato, olive oil, salt, butter, black..."
4,allrecipes.com,Pumpkin Oatmeal,3.9,88,"[rolled oats, milk, pumpkin puree, pumpkin pie..."
...,...,...,...,...,...
94,allrecipes.com,Home-Made Cream of Chicken Soup,3.9,28,"[milk, flour, canola oil, white sugar, finely,..."
95,allrecipes.com,"Broccoli, Rice, Cheese, and Chicken Casserole",4.2,397,"[water, uncooked instant rice, cans chunk chic..."
96,allrecipes.com,Ricotta and Spinach Stuffed Manicotti,4.7,16,"[container ricotta cheese, package spinach, on..."
97,allrecipes.com,Slow-Cooker Chicken Tortilla Soup,4.7,2749,"[can whole peeled tomatoes, can enchilada sauc..."
